<a href="https://colab.research.google.com/github/Pilo1961/Deep_Learning/blob/master/Copia_de_StackGan_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount drive 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# importa modulos propios
import sys
sys.path.append('/content/drive/My Drive/')

import model
#import util

Using TensorFlow backend.


In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage.transform import rescale
from skimage import img_as_ubyte
import pickle
import datetime as dt

Especificamo hiperparametros

In [0]:
data_dir = '/content/drive/My Drive'
train_dir = data_dir + "/train_64"
test_dir = data_dir + "/test_64"
image_size = 64
batch_size = 64
z_dim = 100
stage1_generator_lr = 0.0002
stage1_discriminator_lr = 0.0002
stage1_lr_decay_step = 600
epochs = 1000
condition_dim = 128

#embeddings_file_path_train = train_dir + "/char-CNN-RNN-embeddings.pickle"
#embeddings_file_path_test = test_dir + "/char-CNN-RNN-embeddings.pickle"

#filenames_file_path_train = train_dir + "/filenames.pickle"
#filenames_file_path_test = test_dir + "/filenames.pickle"

#class_info_file_path_train = train_dir + "/class_info.pickle"
#class_info_file_path_test = test_dir + "/class_info.pickle"

#cub_dataset_dir = data_dir + "/CUB_200_2011"

# Load data
## Image list

In [0]:
#esto se debe de ir al modulo util
def add_fileName(df):
  '''
    Add filename column to the ID-description list
  '''
  df['filename']='a'
  for index, row in df.iterrows():
      try:
        new_name = row['ID'][:-6] + '_' + row['ID'][-1] + '.jpg'
        row['filename']=new_name
      except:
        found_n.append(row['ID'])

  return df

def train_test(df):
  x_train = df[df.index % 5 != 0]     # Excludes every 5th row starting from 0
  x_test = df[df.index % 5 == 0]      # Selects every 5th row starting from 0
  #print(x_train.shape)
  #print(x_test.shape)  
  return x_train, x_test


# Load image
def load_image(img_id,src='Flicker8k_Dataset/'):
    I = imread('/content/drive/My Drive/'+src+img_id)
    #I = margin_img(I)
    return I

In [59]:
# Load data
df = pd.read_csv("/content/drive/My Drive/Flickr8k.token.txt", sep='\t', names=['ID', 'Text'], index_col=False)
df=add_fileName(df)
print(df.shape)

# Remove entries with nan values
df.dropna(inplace=True)
df.isnull().values.any()
df.head()

#Quito los registros de una imagen que falta
df.drop([6730,6731,6732,6733,6734],inplace=True)
df.reset_index()
df.shape

(40460, 3)


(40455, 3)

# Embedding

In [0]:
with open("/content/drive/My Drive/embedding_chafa.pkl", 'rb') as f:
  embedding = pickle.load(f) 

In [62]:
print(embedding.shape)
embedding[:5]

(40460, 20)


array([[  0,   0,   0,   2,  42,   3,   2,  89, 171,   6, 118,  50,   2,
        393,  11, 394,   3,  27,   1, 669],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          2,  18, 315,  63,   2, 195, 116],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,
         39,  18, 118,  63,   2, 195,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,  39,
         18, 118,   4, 394,  19,  59,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   2,  39,  18,   3,   2,
         89, 171, 315,  63,   2, 195,   1]], dtype=int32)

In [63]:
df_emb=pd.DataFrame(embedding)
df_emb.head()

#Quito los registros de una imagen que falta
df_emb.drop([6730,6731,6732,6733,6734],inplace=True)
df_emb.reset_index()
df_emb.shape

(40455, 20)

# Train test split

In [64]:
x_train, x_test= train_test(df)
print(x_train.shape)
print(x_test.shape)
x_train.head()

(32364, 3)
(8091, 3)


,ID,Text,filename
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .,1000268201_693b08cb0e_1.jpg
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .,1000268201_693b08cb0e_2.jpg
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...,1000268201_693b08cb0e_3.jpg
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...,1000268201_693b08cb0e_4.jpg
6,1001773457_577c3a7d70.jpg#1,A black dog and a tri-colored dog playing with...,1001773457_577c3a7d70_1.jpg


In [65]:
emb_train, emb_test= train_test(df_emb)
print(emb_train.shape)
print(emb_test.shape)

(32364, 20)
(8091, 20)


# Prepara imagenes

In [0]:
batch_size = 20
dir_train= '/content/drive/My Drive/train_64/'
dir_test= '/content/drive/My Drive/test_64/'

In [0]:
# Hacemos un pickle que tiene un arreglo de numpy con toda la informacion de las imagenes
# ojo:
#file not found:  2258277193_586949ec62.j_1.jpg
#file not found:  2258277193_586949ec62.j_2.jpg
#file not found:  2258277193_586949ec62.j_3.jpg
#file not found:  2258277193_586949ec62.j_4.jpg

def img_train_pickle():
  src='train_64/'
  img_train=[]
  print("Images to load: ", len(x_train["filename"]))
  for i, img_name in enumerate(x_train["filename"]):
    try:
      img=load_image(img_name,src)
      img_train.append(img)
    except:
      print("file not found: ",img_name)
    if i%3000==0: print("Loaded images: ",i)

  with open("/content/drive/My Drive/img_train_data.pkl", 'wb') as f:
    pickle.dump(img_train,f) 

#img_train=np.array(img_train)

In [69]:
# Tenemos las dimensiones correctas
len(img_train)

32364

# Creating models


In [0]:
from keras.optimizers import adam
from keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import concatenate, Embedding, Dense, Dropout, Reshape, UpSampling2D, Conv2D, BatchNormalization, Activation, Input, Concatenate, LeakyReLU, Flatten, Lambda

In [0]:
dis_optimizer = adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
gen_optimizer = adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

In [0]:
def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss

In [0]:
# Esto se debe de ir a models
def create_embedding_compressor(len_embedding=20):
  '''
  Crea la red discrimidaora de la primera etapa
  '''
  input_layer=Input(shape=(len_embedding,))
  emb=Dense(128,activation='relu')(input_layer)
  #out=Reshape((4,4,128),input_shape=(128*4*4,))(emb)
  
  return Model(inputs=[input_layer],outputs=[emb])


In [0]:
ca_model = model.create_CA_model()
ca_model.compile(loss="binary_crossentropy", optimizer="adam")

stage1_dis = model.create_disc_st1()
stage1_dis.compile(loss='binary_crossentropy', optimizer="adam")#dis_optimizer)

stage1_gen = model.create_gen_st1()
stage1_gen.compile(loss="mse", optimizer="adam")#gen_optimizer)

embedding_compressor_model = create_embedding_compressor()
embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")

adversarial_model = model.create_adversarial_model(gen=stage1_gen, disc=stage1_dis)
adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],
                          optimizer="adam", metrics=None)#gen_optimizer, metrics=None)

# Train

Earlier, label/target values for a classifier were 0 or 1; 0 for fake images and 1 for real images. Because of this, GANs were prone to adversarial examples, which are inputs to a neural network that result in an incorrect output from the network. Label smoothing is an approach to provide smoothed labels to the discriminator network. This means we can have decimal values such as 0.9 (true), 0.8 (true), 0.1 (fake), or 0.2 (fake), instead of labeling every example as either 1 (true) or 0 (fake). We smooth the target values (label values) of the real images as well as of the fake images. Label smoothing can reduce the risk of adversarial examples in GANs. To apply label smoothing, assign the labels 0.9, 0.8, and 0.7, and 0.1, 0.2, and 0.3, to the images. To find out more about label smoothing, refer to the following paper.

[Improved techniques for training GAN](https://arxiv.org/pdf/1606.03498.pdf)

In [0]:
real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1

In [93]:
epochs=10
batch_size=64
for epoch in range(epochs):
  print("========================================")
  print("Epoch is:", epoch)
  print("Number of batches", int(img_train.shape[0] / batch_size))

  gen_losses = []
  dis_losses = []

  number_of_batches = int(img_train.shape[0] / batch_size)
  for index in range(number_of_batches):
    print("Batch:{}".format(index+1))

     # Create a batch of noise vectors
    z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
    image_batch = img_train[index * batch_size:(index + 1) * batch_size]
    embedding_batch = emb_train[index * batch_size:(index + 1) * batch_size]
    print("emb sample: ",sum(embedding_batch[1]))
    print("z_noise sample: ",z_noise[1])
    print("imgb sample: ",image_batch[1])

    # Normalize images
    image_batch = (image_batch - 127.5) / 127.5

    fake_images, _ = stage1_gen.predict([embedding_batch, z_noise], verbose=3)
    print(fake_images[1])
    compressed_embedding = embedding_compressor_model.predict_on_batch(embedding_batch)
    print("Salida red compressed embeding ",compressed_embedding.shape)
    compressed_embedding = np.reshape(compressed_embedding, (-1, 1, 1, condition_dim))
    print("reshape salida: ",compressed_embedding.shape)
    compressed_embedding = np.tile(compressed_embedding, (1, 4, 4, 1))
    print("final compressed embedding: ",compressed_embedding[1])

    np.reshape(real_labels, (batch_size, 1))

    dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],np.reshape(real_labels, (batch_size, 1)))
    dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],np.reshape(fake_labels, (batch_size, 1)))
    dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],np.reshape(fake_labels[1:], (batch_size-1, 1)))

    g_loss = adversarial_model.train_on_batch([embedding_batch, z_noise, compressed_embedding],[K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])

    d_loss = 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake))

    print("d_loss:{}".format(d_loss))
            
    print("g_loss:{}".format(g_loss))

    dis_losses.append(d_loss)
    gen_losses.append(g_loss)

Epoch is: 0
Number of batches 505
Batch:1
emb sample:  368
z_noise sample:  [-0.09664526 -0.31356039 -0.22465826 -0.1444633  -1.94868572  0.86167821
  0.6802996  -1.77972119 -1.08934859 -2.31336948  0.1721643   0.194875
 -0.85157603  0.17776879 -1.43694385  0.87035804 -0.48268739 -0.1955283
 -1.35699327 -0.39990885  0.46339748  0.74630552 -0.33317655 -0.81911794
  1.64407317 -0.86127661  1.54318293 -0.59406323 -0.57415176  0.97857379
 -0.09651224  2.19998594 -0.09102715  0.83239422 -1.72393167 -0.4765823
  0.38947615  1.00436622  1.73185639 -1.30329027 -0.55672799 -0.74020204
  1.53430606  1.17837456  0.7306935  -0.74017585  0.87022757  0.30298978
 -0.33960675  2.36779588 -0.02735585 -1.27319223 -0.57193867 -0.8186097
  2.7741256   0.15232765 -0.64692178  0.32817102 -1.31082661  1.06710513
  0.27302203 -0.86994491  0.18626638  0.25024717 -0.70469578 -2.03392706
 -1.54657011 -0.54912944 -0.84570582  1.38821088  0.78482714  1.06770625
 -0.89937267 -0.38775814 -1.77757947  0.00316997  0.0

KeyboardInterrupt: ignored

In [0]:
def save_rgb_img(img, path):
    """
    Save a rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()

In [0]:
z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
embedding_batch = embeddings_test[0:batch_size]
fake_images, _ = stage1_gen.predict_on_batch([embedding_batch, z_noise2])

# Save images
for i, img in enumerate(fake_images[:10]):
    save_rgb_img(img, "results/gen_{}_{}.png".format(epoch, i)

In [0]:
stage1_gen.save_weights("stage1_gen.h5")
stage1_dis.save_weights("stage1_dis.h5")